In [1]:
import pandas as pd
import numpy as np
import re
import string
import emoji
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
import pickle

#for pickle a countvectorizer with lambda function
import dill

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## **Load Training Data**

Wisesight Dataset = 'https://docs.google.com/spreadsheets/d/1F_qT33T2iy0tKbflnVC8Ma-EoWEHimV3NmNRgLjN00o/edit#gid=1302375309'

In [ ]:
df = pd.read_csv('WiseSight Tokenisation Annotation - WiseSight (250 samples_class).csv', sep=',')
df

# **Data Cleaning**

In [ ]:
should_removed = ~df.label.apply(lambda x: len(x.split("-")) > 1)
data_for_training = df[should_removed]
print("we have %d after samples" % len(data_for_training))

In [ ]:
#thai stopwords
thai_stopwords = list(thai_stopwords())

In [ ]:
def clean_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

#ลบคำ เช่น ไวววววววววว ให้เป็น ไว
#(.) means แทนตัวไหนก็ได้
#\1 means first group
#{3,} means ซ้ำมากกว่า 3 ตัว
def clean_rep(text):
    return re.compile(r'(.)\1{3,}', re.IGNORECASE).sub(r'\1', text)

def remove_hashtag(text):
    return re.sub("#(\w+)", '', text)
    
#ลบคำ เช่น ไวไวไวไวไว ให้เป็น ไว
def remove_duplicate(text):
    return re.sub(r'(.+?)\1+', r'\1', text)

def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

def clean_text(text):
    text = text.lower().strip()
    text = clean_url(text)
    text = remove_hashtag(text)
    text = remove_punctuation(text)
    text = clean_rep(text)
    text = remove_duplicate(text)
    text = remove_emoji(text)
    
    #tokenize
    text = word_tokenize(text)
    text = " ".join(word for word in text)
    text = " ".join(word for word in text.split() if word.lower not in thai_stopwords)
    
    return text

In [ ]:
data_for_training['text_tokens'] = data_for_training['raw'].apply(clean_text)
data_for_training

# **Training Count Vectorizor & Sentiment Analyzer**

In [ ]:
#Train, test, split the data

X = data_for_training[['text_tokens']]
y = data_for_training['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#Training CountVetorizer to use for converting a collection of text documents to a matrix of token counts

cvec = CountVectorizer(analyzer=lambda x:x.split(' '))
cvec.fit_transform(X_train['text_tokens'])
cvec.vocabulary_

In [ ]:
train_bow = cvec.transform(X_train['text_tokens'])
pd.DataFrame(train_bow.toarray(), columns=cvec.get_feature_names(), index=X_train['text_tokens'])

In [ ]:
#Using the above BOW to train a LogisticRegression for a Sentiment Analyzer

lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(train_bow, y_train)

In [ ]:
#Test Score

test_bow = cvec.transform(X_test['text_tokens'])
test_predictions = lr.predict(test_bow)
print(classification_report(test_predictions, y_test))

In [ ]:
with open('sentiment_model_pkl', 'wb') as pickle_file:
    pickle.dump(lr, pickle_file)

In [ ]:
with open('countvectorizer_dill', 'wb') as dill_file:
    dill.dump(cvec, dill_file)

# **Testing CountVectorizor & Sentiment Analyzer**

In [ ]:
with open('sentiment_model_pkl', 'rb') as s:
    model = pickle.load(s)

In [ ]:
with open('countvectorizer_dill', 'rb') as c:
    loaded_cvec = dill.load(c)

In [ ]:
def sentiment_prediction(text):
    cleaned_text = clean_text(text)
    bow = loaded_cvec.transform(pd.Series([cleaned_text]))
    prediction = model.predict(bow)[0]
    return prediction

In [ ]:
sentiment_prediction('ไม่เป็นไร เธอทำดีแล้ว')